# Assignment 3

Import libraries and define common helper functions

In [29]:
import os
import sys
import gzip
import json
from pathlib import Path
import csv

import pandas as pd
import s3fs
import pyarrow as pa
from pyarrow.json import read_json
import pyarrow.parquet as pq
import fastavro
import pygeohash
import snappy
import jsonschema
from jsonschema.exceptions import ValidationError


endpoint_url='https://storage.budsc.midwest-datascience.com'

current_dir = Path(os.getcwd()).absolute()
schema_dir = current_dir.joinpath('schemas')
results_dir = current_dir.joinpath('results')
results_dir.mkdir(parents=True, exist_ok=True)


def read_jsonl_data():
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    with s3.open(src_data_path, 'rb') as f_gz:
        with gzip.open(f_gz, 'rb') as f:
            records = [json.loads(line) for line in f.readlines()]
        

    return records

In [ ]:
Create a JSON Schema in the schemas/routes-schema.json file to describe a route and validate the data in routes.jsonl.gz
using the jsonschema library.



Load the records from https://storage.budsc.midwest-datascience.com/data/processed/openflights/routes.jsonl.gz 

In [30]:
records = read_jsonl_data()

In [38]:
records[0:3]

[{'airline': {'airline_id': 410,
   'name': 'Aerocondor',
   'alias': 'ANA All Nippon Airways',
   'iata': '2B',
   'icao': 'ARD',
   'callsign': 'AEROCONDOR',
   'country': 'Portugal',
   'active': True},
  'src_airport': {'airport_id': 2965,
   'name': 'Sochi International Airport',
   'city': 'Sochi',
   'country': 'Russia',
   'iata': 'AER',
   'icao': 'URSS',
   'latitude': 43.449902,
   'longitude': 39.9566,
   'altitude': 89,
   'timezone': 3.0,
   'dst': 'N',
   'tz_id': 'Europe/Moscow',
   'type': 'airport',
   'source': 'OurAirports'},
  'dst_airport': {'airport_id': 2990,
   'name': 'Kazan International Airport',
   'city': 'Kazan',
   'country': 'Russia',
   'iata': 'KZN',
   'icao': 'UWKD',
   'latitude': 55.606201171875,
   'longitude': 49.278701782227,
   'altitude': 411,
   'timezone': 3.0,
   'dst': 'N',
   'tz_id': 'Europe/Moscow',
   'type': 'airport',
   'source': 'OurAirports'},
  'codeshare': False,
  'equipment': ['CR2']},
 {'airline': {'airline_id': 410,
   'nam

In [42]:
df = pd.DataFrame(records)
table = pa.Table.from_pandas(df)

pq.write_table(table,'routes.parquet')

## 3.1

### 3.1.a JSON Schema

In [27]:
def validate_jsonl_data(records):
    schema_path = 'routes-schema.json'
    with open(schema_path) as f:
        schema = json.load(f)
        
#    with open(validation_csv_path, 'w') as f:    
    for i, record in enumerate(records):
        try:
            jsonschema.validate(instance=records[1], schema=schema)
            pass
        except ValidationError as e:
            print("The schema doesn't fit for this record")
            pass
            

validate_jsonl_data(records)

In [33]:
schema_avro = {   "doc": "route.avsc",
    "name": "route_avsc",
    "namespace": "route_avro",
    "type": "record",
    "fields": [
       { "name": "airline_id","type": "int","default" : "NONE"},
        {"name": "name","type": "string"},
        {"name": "alias","type": "string"},
        {"name":"iata",
          "type": "string"
        },
        {"name":"icao",
          "type": "string"
        },
        {"name":"callsign",
          "type": "string"
        },
        {"name":"country",
          "type": "string"
        },
        {"name":"active",
          "type": "boolean"
        }
        ,
     {"name":"src_airport",
      "type": {
            "type": "record",
            "name": "src_airport",
            "fields": [
        {"name":"airport_id",
          "type": "int"
        },
        {"name":"name",
          "type": "string"
        },
        {"name":"city",
          "type": "string"
        },
        {"name":"country",
          "type": "string"
        },
        {"name":"iata",
          "type": "string"
        },
        {"name":"icao",
          "type": "string"
        },
        {"name":"latitude",
          "type": "long"
        },
        {"name":"longitude",
          "type": "long"
        },
        {"name":"altitude",
          "type": "int"
        },
        {"name":"timezone",
          "type": "long"
        },
        {"name":"dst",
          "type": "string"
        },
        {"name":"tz_id",
          "type": "string"
        },
        {"name":"type",
          "type": "string"
        },
        {"name":"source",
          "type": "string"
        }
      ]
     }
	 },
    { "name":"dst_airport",
     "type": {
            "type": "record",
            "name": "dst_airport",
            "fields": [
        {"name":"airport_id",
          "type": "int"
        },
        {"name":"name",
          "type": "string"
        },
        {"name":"city",
          "type": "string"
        },
        {"name":"country",
          "type": "string"
        },
        {"name":"iata",
          "type": "string"
        },
        {"name":"icao",
          "type": "string"
        },
        {"name":"latitude",
          "type": "long"
        },
        {"name":"longitude",
          "type": "long"
        },
        {"name":"altitude",
          "type": "int"
        },
        {"name":"timezone",
          "type": "long"
        },
        {"name":"dst",
          "type": "string"
        },
        {"name":"tz_id",
          "type": "string"
        },
        {"name":"type",
          "type": "string"
        },
        {"name":"source",
          "type": "string"
        }
      ]
    }
	},
    {"name":"codeshare",
      "type": "boolean"
    },
    {"name":"equipment",
      "type": {
            "type": "record",
            "name": "equipment",
			"fields": [
				{
					
                                         "name" : "code",  
                                         "type": "string"
				}
						]
				}
			}
]
}


In [43]:
#Testing avrso schema
from fastavro import parse_schema
from fastavro import writer

parsed_schema = parse_schema(schema_avro)
with open('routes.avsc', 'wb') as out:
    writer(out, parsed_schema, records)

### 3.1.b Avro

In [ ]:
def create_avro_dataset(records):
    schema_path = schema_dir.joinpath('routes.avsc')
    data_path = results_dir.joinpath('routes.avro')
    ## TODO: Use fastavro to create Avro dataset
    parsed_schema = parse_schema(schema_avro)
    with open('routes.avro', 'wb') as out:
        writer(out, parsed_schema, records[0:2])
    
        
create_avro_dataset(records)

### 3.1.c Parquet

In [46]:
def create_parquet_dataset():
    src_data_path = 'data/processed/openflights/routes.jsonl.gz'
    parquet_output_path = results_dir.joinpath('routes.parquet')
    s3 = s3fs.S3FileSystem(
        anon=True,
        client_kwargs={
            'endpoint_url': endpoint_url
        }
    )
    df = pd.DataFrame(records)
    table = pa.Table.from_pandas(df)

    pq.write_table(table,'routes.parquet')
    
#     with s3.open(src_data_path, 'rb') as f_gz:
#         with gzip.open(f_gz, 'rb') as f:
#             df = pd.pandas(f)
#             table = pa.Table.from_pandas(df)
#             print(table)
#             pass
#             ## TODO: Use Apache Arrow to create Parquet table and save the dataset

create_parquet_dataset()

### 3.1.d Protocol Buffers

In [48]:
import routes_pb2
def _airline_to_proto_obj(airline):
    obj = routes_pb2.Airline()
    print(obj)
    if not airline.get('name'):
        return None
    if not airline.get('airline_id'):
        return None
    
    obj.airport_id = airport.get('airport_id')
    if airport.get('name'):
        obj.name = airport.get('name')
    if airport.get('city'):
        obj.city = airport.get('city')
    if airport.get('iata'):
        obj.iata = airport.get('iata')
    if airport.get('icao'):
        obj.icao = airport.get('icao')
    if airport.get('altitude'):
        obj.altitude = airport.get('altitude')
    if airport.get('timezone'):
        obj.timezone = airport.get('timezone')
    if airport.get('dst'):
        obj.dst = airport.get('dst')
    if airport.get('tz_id'):
        obj.tz_id = airport.get('tz_id')
    if airport.get('type'):
        obj.type = airport.get('type')
    if airport.get('source'):
        obj.source = airport.get('source')

    obj.latitude = airport.get('latitude')
    obj.longitude = airport.get('longitude')

    return obj

def create_protobuf_dataset(records):
    routes = routes_pb2.Routes()
    for record in records:
        route = routes_pb2.Route()
        airline = _airline_to_proto_obj(record.get('airline', {}))
        print(airline)
        if airline:
            route.airline.CopyFrom(airline)
        #TODO
        routes.route.append(route)
    data_path = results_dir.joinpath('routes.pb')
    with open(data_path, 'wb') as f:
        f.write(routes.SerializeToString())
    compressed_path = results_dir.joinpath('routes.pb.snappy')
    with open(compressed_path, 'wb') as f:
        f.write(snappy.compress(routes.SerializeToString()))

In [49]:
create_protobuf_dataset(records)

AttributeError: 'Airline' object has no attribute 'airport_id'

## 3.2

### 3.2.a Simple Geohash Index

In [51]:
def create_hash_dirs(records):
    geoindex_dir = results_dir.joinpath('geoindex')
    geoindex_dir.mkdir(exist_ok=True, parents=True)
    hashes = []
    for record in records:
        src_airport = record.get('src_airport', {})
        if src_airport:
            latitude = src_airport.get('latitude')
            longitude = src_airport.get('longitude')
            if latitude and longitude:
                #TODO: use pygeohash.encode() to assign geohashes to the records and complete the hashes list
                h = pygeohash.encode(latitude,longitude)
                record['geohash']=h
                hashes.append(h)
    hashes.sort()
    three_letter = sorted(list(set([entry[:3] for entry in hashes])))
    hash_index = {value: [] for value in three_letter}
    for record in records:
        geohash = record.get('geohash')
        if geohash:
            hash_index[geohash[:3]].append(record)
    for key, values in hash_index.items():
        output_dir = geoindex_dir.joinpath(str(key[:1])).joinpath(str(key[:2]))
        output_dir.mkdir(exist_ok=True, parents=True)
        output_path = output_dir.joinpath('{}.jsonl.gz'.format(key))
        with gzip.open(output_path, 'w') as f:
            json_output = '\n'.join([json.dumps(value) for value in values])
            f.write(json_output.encode('utf-8'))
create_hash_dirs(records)

Implement a simple geospatial search feature that finds airports within a specified distance of an input latitude 
and longitude.
You can use the geohash_approximate_distance function in pygeohash to compute distances between geohash values. 
It returns distances in meters, but your search function should use kilometers as input.


import pygeohash
pygeohash.geohash_approximate_distance('bcd3u', 'bc83n')
625441


### 3.2.b Simple Search Feature

In [52]:
def airport_search(latitude, longitude):
    ## TODO: Create simple search to return nearest airport
    h = pygeohash.encode(latitude,longitude)
    dist = 0
    name = ''
    for i,record in enumerate(records):
        src_airport = record.get('src_airport', {})
        if src_airport:
            lat = src_airport.get('latitude')
            long = src_airport.get('longitude')
            a_name = src_airport.get('name')
            if lat and long:
                h1 = pygeohash.encode(lat,long)
                
                dist_n = pygeohash.geohash_approximate_distance(h,h1)
                if i==0:
                    dist = dist_n
                else:
                    if dist > dist_n:
                        dist = dist_n
                        name = a_name
    print(name)
                        
                
                    

    
#airport_search(41.1499988, -95.91779)
airport_search(41.1499988, -95.91779)

Eppley Airfield


In [53]:
#Astrakhan 
airport_search(46.2832984924,48.0063018799)

Astrakhan Airport
